I'll be checking for multi-collinearity in the ASCT dataset to ensure that the cox regression model is appropiately fitted. 

The continous variables are: 
1. Age 
2. Hemoglobin concentration 
3. Creatinine concentration 

The categorical variables are: 
1. Gender
2. At risk (Y/N) 
3. Tandem (Y/N) 
4. Number of maintenace drugs (dummy variable for 1) 
5. Number of maintenace drugs (dummy variable for 2) 

In this notebook, I'll be assesing the categorical variables with a log-linear model

Helpful links: 

- Implenting a log-linear model: https://data.library.virginia.edu/an-introduction-to-loglinear-models/
- Converting between a data frame of cases, a data frame of counts of each type of case, and a contingency table : http://www.cookbook-r.com/Manipulating_data/Converting_between_data_frames_and_contingency_tables/#cases-to-counts

Load Necessary libraries 

In [1]:
#Import libraries here 

Load dataset

In [2]:
ASCT_data <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Mina/MM_ACST_Study/Prepared_data_for_R_analysis/ASCT_Multivar.csv')


ASCT_data <- transform(ASCT_data, Creatinine = as.numeric(Creatinine))
#head(ASCT_data)

Create new df with only categorical variables

In [3]:
df_cat <- ASCT_data[,c("GENDER", "At_Risk..Y.N..for.Anthony", "TANDEM",'Num..Maint..Drugs...Dummy.var.for.1.','Num..Maint..Drugs...Dummy.var.for.2.' )]
#head(df_cat)

Converting dataframe of cases to a dataframe of counts

In [4]:
countdf <- as.data.frame(table(df_cat))
countdf

GENDER,At_Risk..Y.N..for.Anthony,TANDEM,Num..Maint..Drugs...Dummy.var.for.1.,Num..Maint..Drugs...Dummy.var.for.2.,Freq
0,0,0,0,0,31
1,0,0,0,0,42
0,1,0,0,0,5
1,1,0,0,0,11
0,0,1,0,0,0
1,0,1,0,0,1
0,1,1,0,0,5
1,1,1,0,0,7
0,0,0,1,0,6
1,0,0,1,0,10


First model: Assuming all are independent 

In [5]:
#fitting the model and displaying the summary 
mod0 <- glm(Freq ~ GENDER + At_Risk..Y.N..for.Anthony + TANDEM + Num..Maint..Drugs...Dummy.var.for.1. + Num..Maint..Drugs...Dummy.var.for.2., 
            data = countdf, family = poisson)
summary(mod0)

#displaying the p-value of the model
pchisq(deviance(mod0), df = df.residual(mod0), lower.tail = F)
print("Since the null of this test is that the expected frequencies satisfy the given loglinear model, clearly they do not (since p < 0.05 and hence you reject the null). ")


Call:
glm(formula = Freq ~ GENDER + At_Risk..Y.N..for.Anthony + TANDEM + 
    Num..Maint..Drugs...Dummy.var.for.1. + Num..Maint..Drugs...Dummy.var.for.2., 
    family = poisson, data = countdf)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.2077  -1.3813  -0.4961   0.7177   3.9422  

Coefficients:
                                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)                             3.0161     0.1576  19.132  < 2e-16 ***
GENDER1                                 0.3947     0.1643   2.402   0.0163 *  
At_Risk..Y.N..for.Anthony1             -0.4217     0.1648  -2.560   0.0105 *  
TANDEM1                                -1.3782     0.2010  -6.858 6.99e-12 ***
Num..Maint..Drugs...Dummy.var.for.1.1  -1.0473     0.1838  -5.699 1.20e-08 ***
Num..Maint..Drugs...Dummy.var.for.2.1  -2.4709     0.3006  -8.220  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

[1] 2.36877e-10

[1] "Since the null of this test is that the expected frequencies satisfy the given loglinear model, clearly they do not (since p < 0.05 and hence you reject the null). "


Compare fitted values to the observed values 

In [6]:
cbind(mod0$data, fitted(mod0))
print("Evidently the fit could be better")

GENDER,At_Risk..Y.N..for.Anthony,TANDEM,Num..Maint..Drugs...Dummy.var.for.1.,Num..Maint..Drugs...Dummy.var.for.2.,Freq,fitted(mod0)
0,0,0,0,0,31,20.4122116
1,0,0,0,0,42,30.2890882
0,1,0,0,0,5,13.3886549
1,1,0,0,0,11,19.8670363
0,0,1,0,0,0,5.1445411
1,0,1,0,0,1,7.6338352
0,1,1,0,0,5,3.3743764
1,1,1,0,0,7,5.0071392
0,0,0,1,0,6,7.1621795
1,0,0,1,0,10,10.6277502


[1] "Evidently the fit could be better"


Second model: homogeneous association ("pairwise interactions") 

In [7]:
#fitting the model and displaying the summary 
mod1 <- glm(Freq ~ (GENDER + At_Risk..Y.N..for.Anthony + TANDEM + Num..Maint..Drugs...Dummy.var.for.1. + Num..Maint..Drugs...Dummy.var.for.2.)^2, 
            data = countdf, family = poisson)
summary(mod1)


#displaying the p-value of the model
pchisq(deviance(mod1), df = df.residual(mod1), lower.tail = F)
print("The high p-value says we have insufficient evidence to reject the null hypothesis that the expected frequencies satisfy our model.")


Call:
glm(formula = Freq ~ (GENDER + At_Risk..Y.N..for.Anthony + TANDEM + 
    Num..Maint..Drugs...Dummy.var.for.1. + Num..Maint..Drugs...Dummy.var.for.2.)^2, 
    family = poisson, data = countdf)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.40036  -0.54867  -0.00015   0.28775   1.72849  

Coefficients:
                                                                              Estimate
(Intercept)                                                                    3.34251
GENDER1                                                                        0.41213
At_Risk..Y.N..for.Anthony1                                                    -1.26166
TANDEM1                                                                       -3.36220
Num..Maint..Drugs...Dummy.var.for.1.1                                         -1.37162
Num..Maint..Drugs...Dummy.var.for.2.1                                         -4.82257
GENDER1:At_Risk..Y.N..for.Anthony1                   

[1] 0.5143273

[1] "The high p-value says we have insufficient evidence to reject the null hypothesis that the expected frequencies satisfy our model."


Compare fitted vlaues to the observed values 

In [8]:
cbind(mod1$data, fitted(mod1))
print("Evidently the fit is good")

GENDER,At_Risk..Y.N..for.Anthony,TANDEM,Num..Maint..Drugs...Dummy.var.for.1.,Num..Maint..Drugs...Dummy.var.for.2.,Freq,fitted(mod1)
0,0,0,0,0,31,2.828998e+01
1,0,0,0,0,42,4.271876e+01
0,1,0,0,0,5,8.011289e+00
1,1,0,0,0,11,9.979966e+00
0,0,1,0,0,0,9.805023e-01
1,0,1,0,0,1,2.010752e+00
0,1,1,0,0,5,3.718226e+00
1,1,1,0,0,7,6.290519e+00
0,0,0,1,0,6,7.177038e+00
1,0,0,1,0,10,1.016507e+01


[1] "Evidently the fit is good"


Note that by implementing these models, you can also retrieve the odds ratios of how categorical variables interact. For more details, check the first link which I tagged above. 

Third model: modelling the 5-way interactions 
- (this is extra since modelling pairwise interactions was sufficient)

In [9]:
#fitting the model and displaying the summary 
mod2 <- glm(Freq ~ GENDER * At_Risk..Y.N..for.Anthony * TANDEM * Num..Maint..Drugs...Dummy.var.for.1. * Num..Maint..Drugs...Dummy.var.for.2., 
            data = countdf, family = poisson)
summary(mod2)

#displaying the p-value of the model
pchisq(deviance(mod2), df = df.residual(mod2), lower.tail = F)


Call:
glm(formula = Freq ~ GENDER * At_Risk..Y.N..for.Anthony * TANDEM * 
    Num..Maint..Drugs...Dummy.var.for.1. * Num..Maint..Drugs...Dummy.var.for.2., 
    family = poisson, data = countdf)

Deviance Residuals: 
 [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
[26]  0  0  0  0  0  0  0

Coefficients:
                                                                                                                         Estimate
(Intercept)                                                                                                             3.434e+00
GENDER1                                                                                                                 3.037e-01
At_Risk..Y.N..for.Anthony1                                                                                             -1.825e+00
TANDEM1                                                                                                                -2.874e+01
Num..Maint..

[1] 0

Compare fitted values to the observed values 

In [10]:
cbind(mod2$data, fitted(mod2))

GENDER,At_Risk..Y.N..for.Anthony,TANDEM,Num..Maint..Drugs...Dummy.var.for.1.,Num..Maint..Drugs...Dummy.var.for.2.,Freq,fitted(mod2)
0,0,0,0,0,31,3.100000e+01
1,0,0,0,0,42,4.200000e+01
0,1,0,0,0,5,5.000000e+00
1,1,0,0,0,11,1.100000e+01
0,0,1,0,0,0,1.026188e-11
1,0,1,0,0,1,1.000000e+00
0,1,1,0,0,5,5.000000e+00
1,1,1,0,0,7,7.000000e+00
0,0,0,1,0,6,6.000000e+00
1,0,0,1,0,10,1.000000e+01


We can verify that the homogeneous association model fits just as well as the saturated model by performing a likelihood ratio test. One way to do this is with the anova function:

In [11]:
anova(mod1, mod2)

Resid. Df,Resid. Dev,Df,Deviance
16,1.514113e+01,NA,NA
0,2.873220e-10,16,15.14113


In [12]:
pchisq(15.14, df = 16, lower.tail = F)
print("This says we fail to reject the null hypothesis that mod1 fits just as well as mod2.")

[1] 0.5144096

[1] "This says we fail to reject the null hypothesis that mod1 fits just as well as mod2."
